In [1]:
import os
import sys, traceback, time

from datetime import datetime, date

from decimal import Decimal

from sqlalchemy import create_engine, MetaData, select
from sqlalchemy.orm import sessionmaker
from pymongo import MongoClient

# Configuración de la conexión a PostgreSQL
postgres_username = 'alumnodb'
postgres_password = 'alumnodb'
postgres_host = 'localhost'
postgres_port = '5432'
postgres_database = 'si1'

# Configuración de la conexión a MongoDB
mongodb_host = 'localhost'
mongodb_port = 27017
mongodb_database = 'si1'
mongodb_collection = 'france'


In [2]:
# Conexión a PostgreSQL
postgres_uri = f'postgresql://{postgres_username}:{postgres_password}@{postgres_host}:{postgres_port}/{postgres_database}'
postgres_engine = create_engine(postgres_uri)
Session = sessionmaker(bind=postgres_engine)
session = Session()

# Obtener metadatos de las tablas en PostgreSQL
metadata = MetaData()
metadata.reflect(bind=postgres_engine)

# Connect to MongoDB
mongodb_client = MongoClient(mongodb_host, mongodb_port)
mongodb_db = mongodb_client[mongodb_database]

In [6]:
metadata.tables.keys()

dict_keys(['imdb_actormovies', 'imdb_actors', 'imdb_movies', 'imdb_moviecountries', 'inventory', 'customers', 'imdb_directors', 'imdb_moviegenres', 'imdb_movielanguages', 'orders', 'orderdetail', 'imdb_directormovies', 'products'])

In [26]:
tables_to_process = ['imdb_actormovies', 'imdb_actors', 'imdb_movies', 'imdb_moviecountries', 'imdb_directors', 'imdb_moviegenres', 'imdb_directormovies']
movies = []
actors = []
actormovies = []
moviecountries = []
directors = []
directormovies = []
moviegenres = []
for table_name, table in metadata.tables.items():
    if table_name not in tables_to_process:
        continue
    print(f'Processing {table_name}...')
    records = session.query(table).all()
    if table_name == 'imdb_actors':
        for record in records:
            actors.append({'actorid': record._asdict().get('actorid'), 'actorname': record._asdict().get('actorname')})
    elif table_name == 'imdb_movies':
        for record in records:
            title = record._asdict().get('movietitle')
            if title is not None:
                movietitle = title.rsplit('(', 1)[0].strip()
            else:
                movietitle = None
            movies.append({'movieid': record._asdict().get('movieid'), 'movietitle': movietitle, 'year': record._asdict().get('year')})
    elif table_name == 'imdb_actormovies':
        for record in records: 
            actormovies.append({'actorid': record._asdict().get('actorid'), 'movieid': record._asdict().get('movieid')})
    elif table_name == 'imdb_moviecountries':
        for record in records:
            moviecountries.append({'movieid': record._asdict().get('movieid'), 'country': record._asdict().get('country')})
    elif table_name == 'imdb_directors':
        for record in records:
            directors.append({'directorid': record._asdict().get('directorid'), 'directorname': record._asdict().get('directorname')})
    elif table_name == 'imdb_directormovies':
        for record in records:
            directormovies.append({'directorid': record._asdict().get('directorid'), 'movieid': record._asdict().get('movieid')})
    elif table_name == 'imdb_moviegenres':
        for record in records:
            moviegenres.append({'movieid': record._asdict().get('movieid'), 'genre': record._asdict().get('genre')})
    else:
        print(f'Unknown table {table_name}')
        continue
    print(f'{table_name} processed.')    
    
      

Processing imdb_actormovies...
imdb_actormovies processed.
Processing imdb_actors...
imdb_actors processed.
Processing imdb_movies...
imdb_movies processed.
Processing imdb_moviecountries...
imdb_moviecountries processed.
Processing imdb_directors...
imdb_directors processed.
Processing imdb_moviegenres...
imdb_moviegenres processed.
Processing imdb_directormovies...
imdb_directormovies processed.


In [32]:
from IPython.display import display, clear_output

mongodb_movies = []

for movie in movies:    
    clear_output(wait=True)
    display(f'Progress: {movies.index(movie) + 1}/{len(movies)}')
    movie_genres = [item['genre'] for item in moviegenres if item['movieid'] == movie['movieid']]
    
    movie_directors = [item['directorname'] for item in directors if item['directorid'] in [dm['directorid'] for dm in directormovies if dm['movieid'] == movie['movieid']]]
    
    # Faster way to get actors
    actormovies_dict = {}
    for am in actormovies:
        if am['movieid'] == movie['movieid']:
            if am['movieid'] not in actormovies_dict:
                actormovies_dict[am['movieid']] = set()
            actormovies_dict[am['movieid']].add(am['actorid'])

    movie_actors = [item['actorname'] for item in actors if item['actorid'] in actormovies_dict.get(movie['movieid'], set())]
    
    most_related_movies = []
    if len(movie_genres) > 1:
        for other_movie in movies:
            other_movie_genres = [item['genre'] for item in moviegenres if item['movieid'] == other_movie['movieid']]
            if set(movie_genres) == set(other_movie_genres) and other_movie['movieid'] != movie['movieid']:
                most_related_movies.append({'title': other_movie['movietitle'], 'year': other_movie['year']})
        most_related_movies = sorted(most_related_movies, key=lambda x: x['year'], reverse=True)[:10]

    related_movies = []
    if len(movie_genres) > 1:
        for other_movie in movies:
            other_movie_genres = [item['genre'] for item in moviegenres if item['movieid'] == other_movie['movieid']]
            if len(set(movie_genres).intersection(set(other_movie_genres))) / len(set(movie_genres)) >= 0.5 and other_movie['movieid'] != movie['movieid'] and other_movie not in most_related_movies:
                related_movies.append({'title': other_movie['movietitle'], 'year': other_movie['year']})

    mongodb_movies.append({
        'title': movie['movietitle'],
        'genres': movie_genres,
        'year': movie['year'],
        'directors': movie_directors,
        'actors': movie_actors,
        'most_related_movies': most_related_movies,
        'related_movies': related_movies
    })

'Progress: 296/3873'

KeyboardInterrupt: 

In [12]:
from collections import defaultdict
from IPython.display import display, clear_output

# Define a dictionary to store data instead of separate lists
data = {
    'movies': [],
    'actors': [],
    'actormovies': [],
    'moviecountries': [],
    'directors': [],
    'directormovies': [],
    'moviegenres': [],
}

tables_to_process = ['imdb_actormovies', 'imdb_actors', 'imdb_movies', 'imdb_moviecountries', 'imdb_directors', 'imdb_moviegenres', 'imdb_directormovies']

# Collect data in a dictionary instead of separate lists
for table_name, table in metadata.tables.items():
    if table_name not in tables_to_process:
        continue
    print(f'Processing {table_name}...')
    records = session.query(table).all()
    if table_name == 'imdb_actors':
        data['actors'] = [{'actorid': record.actorid, 'actorname': record.actorname} for record in records]
    elif table_name == 'imdb_movies':
        data['movies'] = [{'movieid': record.movieid, 'movietitle': record.movietitle.rsplit('(', 1)[0].strip(), 'year': record.year} for record in records if record.movietitle is not None]
    elif table_name == 'imdb_actormovies':
        data['actormovies'] = [{'actorid': record.actorid, 'movieid': record.movieid} for record in records]
    elif table_name == 'imdb_moviecountries':
        data['moviecountries'] = [{'movieid': record.movieid, 'country': record.country} for record in records]
    elif table_name == 'imdb_directors':
        data['directors'] = [{'directorid': record.directorid, 'directorname': record.directorname} for record in records]
    elif table_name == 'imdb_directormovies':
        data['directormovies'] = [{'directorid': record.directorid, 'movieid': record.movieid} for record in records]
    elif table_name == 'imdb_moviegenres':
        data['moviegenres'] = [{'movieid': record.movieid, 'genre': record.genre} for record in records]
    else:
        print(f'Unknown table {table_name}')
        continue
    print(f'{table_name} processed.')

# Convert actormovies to a dictionary for faster access
actormovies_dict = defaultdict(set)
for am in data['actormovies']:
    actormovies_dict[am['movieid']].add(am['actorid'])

# Process movies
mongodb_movies = []

for movie in data['movies']:
    # clear_output(wait=True)
    display(f'Progress: {data["movies"].index(movie) + 1}/{len(data["movies"])}')
    
    country_records = [item['country'] for item in data['moviecountries'] if item['movieid'] == movie['movieid']]
    if 'France' not in country_records:
        continue

    movie_genres = [item['genre'] for item in data['moviegenres'] if item['movieid'] == movie['movieid']]
    movie_directors = [item['directorname'] for item in data['directors'] if item['directorid'] in [dm['directorid'] for dm in data['directormovies'] if dm['movieid'] == movie['movieid']]]
    movie_actors = [item['actorname'] for item in data['actors'] if item['actorid'] in actormovies_dict.get(movie['movieid'], set())]

    most_related_movies = []
    if len(movie_genres) >= 1:
        for other_movie in data['movies']:
            other_movie_genres = [item['genre'] for item in data['moviegenres'] if item['movieid'] == other_movie['movieid']]
            if set(movie_genres) == set(other_movie_genres) and other_movie['movieid'] != movie['movieid']:
                most_related_movies.append({'title': other_movie['movietitle'], 'year': other_movie['year']})
        most_related_movies = sorted(most_related_movies, key=lambda x: x['year'], reverse=True)[:10]

    related_movies = []
    if len(movie_genres) > 1:
        for other_movie in data['movies']:
            other_movie_genres = [item['genre'] for item in data['moviegenres'] if item['movieid'] == other_movie['movieid']]
            if len(set(movie_genres).intersection(set(other_movie_genres))) / len(set(movie_genres)) >= 0.5 and other_movie['movieid'] != movie['movieid'] and other_movie not in most_related_movies:
                related_movies.append({'title': other_movie['movietitle'], 'year': other_movie['year']})

    mongodb_movies.append({
        'title': movie['movietitle'],
        'genres': movie_genres,
        'year': movie['year'],
        'directors': movie_directors,
        'actors': movie_actors,
        'most_related_movies': most_related_movies,
        'related_movies': related_movies
    })
    print(f'title: {movie["movietitle"]}\n genres: {movie_genres}\n year: {movie["year"]}\n directors: {movie_directors}\n actors: {movie_actors}\n most_related_movies: {most_related_movies}\n related_movies: {related_movies}')


Processing imdb_actormovies...
imdb_actormovies processed.
Processing imdb_actors...
imdb_actors processed.
Processing imdb_movies...
imdb_movies processed.
Processing imdb_moviecountries...
imdb_moviecountries processed.
Processing imdb_directors...
imdb_directors processed.
Processing imdb_moviegenres...
imdb_moviegenres processed.
Processing imdb_directormovies...
imdb_directormovies processed.


'Progress: 1/3873'

'Progress: 2/3873'

title: Ran
 genres: ['Action', 'Drama']
 year: 1985
 directors: ['Kurosawa, Akira']
 actors: ['Amada, Masuo', 'Igawa, Hisashi', 'Ito, Toshiya', 'Kato, Kazuo (I)', 'Katô, Takeshi (I)', 'Kawai, Hanbei', 'Kimura, Sakae', 'Kodama, Kenji (I)', 'Matsui, Norio', 'Nagasawa, Ryo', 'Nakadai, Tatsuya', 'Nezu, Jinpachi', 'Nomura, Mansai', 'Oki, Ryojiro', 'Peter (I)', 'Ryu, Daisuke', 'Sakurai, Masaru', 'Sasaki, Masaaki', 'Sugizaki, Akihiko', 'Suzuki, Heihachiro', 'Tazaki, Jun', 'Terao, Akira', 'Ueki, Hitoshi', 'Watanabe, Takashi (V)', 'Yamaguchi, Yoshimitsu', 'Yamashita, Tetsuo', 'Yui, Masayuki', 'Zushi, Takao', 'Zushi, Yoshitaka', 'Harada, Mieko', 'Kanda, Tokie', 'Kochi, Sawako', 'Miyazaki, Yoshiko', 'Nanjo, Reiko', 'Otowa, Kumeko', 'Togo, Haruko']
 most_related_movies: [{'title': 'Men of Means', 'year': '1999'}, {'title': "In God's Hands", 'year': '1998'}, {'title': 'G.I. Jane', 'year': '1997'}, {'title': 'Iron Eagle IV', 'year': '1995'}, {'title': 'Sudden Death', 'year': '1995'}, {'title': 'Dung

'Progress: 3/3873'

...

title: Amants criminels, Les
 genres: ['Crime', 'Drama', 'Thriller']
 year: 1999
 directors: ['Ozon, François']
 actors: ['Debard, Jean-Louis', 'Demurger, Gil', 'Deschanel, Guillaume', 'Desolière, Patrice', 'Doignon, Didier', 'Drumelle, Philippe', 'Durand, Guy', 'Hamdaoui, Rabah', 'Adjeri, Ranzi', 'Albuquerque, Martihino', 'Albuquerque, Nicolas', 'Belmadi, Yasmine', 'Boyer, Aurélien', 'Bozzo, Jean-Christophe', 'Brolis, Sébastien', 'Brunéo, Laurent', 'Cathelin, Lionel', 'Da Silva, Fernand', 'Dalle, Jerôme', 'Jallut, Julien', 'Kechiouche, Salim', 'Lamadon, Ludovic', 'Latrasse, Guillaume', 'Lauret, François', 'Lemoine, Thierry', 'Luiz, Christophe', 'Mallet, Renaud', 'Manojlovic, Miki', 'Martres, Julien', 'Maume, Bernard', 'Morano, Renaud', 'Moresco, Alain', 'Papot, Olivier', 'Piano, Jonathan', 'Picot, Frédéric', 'Renier, Jérémie', 'Teissière, Arnaud', 'Accaria, Agnès', 'Albuquerque, Brigitte', 'Aubert, Myriam', 'Borel, Anne-Laure', 'Chazot, Emmanuelle', 'Colas, Sylvie', 'Coubaillon, Marie

'Progress: 128/3873'

title: Amants du Pont-Neuf, Les
 genres: ['Drama', 'Romance']
 year: 1991
 directors: ['Carax, Leos']
 actors: ['Desclozeaux, Marc', 'Grüber, Klaus-Michael', 'Airola, Jean-Louis', 'Aldama, Johnny', 'Aperghis, Georges', 'Berthonnier, Roger', 'Bitoun, Maître', 'Buain, Daniel', 'Castorp, Georges', 'Dahan, Alain (I)', 'Larsson, Chrichan', 'Lavant, Denis', 'Maurette, Marc', 'Pessemesse, Pierre', 'Prévost, Albert', 'Vandestien, Michel', 'Berthonnier, Paulette', 'Binoche, Juliette', 'Scob, Edith', 'Stalens, Marion', 'Trintignant, Marie']
 most_related_movies: [{'title': 'Autumn in New York', 'year': '2000'}, {'title': 'Golden Bowl, The', 'year': '2000'}, {'title': 'Here on Earth', 'year': '2000'}, {'title': 'Where the Heart Is', 'year': '2000'}, {'title': 'At First Sight', 'year': '1999'}, {'title': 'Cider House Rules, The', 'year': '1999'}, {'title': 'Dreaming of Joseph Lees', 'year': '1999'}, {'title': 'Floating', 'year': '1999'}, {'title': 'Guinevere', 'year': '1999'}, {'title': 'Message i

'Progress: 129/3873'

KeyboardInterrupt: 

In [11]:
from collections import defaultdict
from IPython.display import display, clear_output

# Define a dictionary to store data instead of separate lists
data = {
    'movies': [],
    'actors': [],
    'actormovies': [],
    'moviecountries': [],
    'directors': [],
    'directormovies': [],
    'moviegenres': [],
}

tables_to_process = ['imdb_actormovies', 'imdb_actors', 'imdb_movies', 'imdb_moviecountries', 'imdb_directors', 'imdb_moviegenres', 'imdb_directormovies']

for table_name, table in metadata.tables.items():
    if table_name == 'imdb_moviecountries':
        data['moviecountries'] = [{'movieid': record.movieid, 'country': record.country} for record in records]

# Collect data in a dictionary instead of separate lists
for table_name, table in metadata.tables.items():
    if table_name not in tables_to_process:
        continue
    print(f'Processing {table_name}...')
    records = session.query(table).all()
    if table_name == 'imdb_actors':
        data['actors'] = [{'actorid': record.actorid, 'actorname': record.actorname} for record in records]
    elif table_name == 'imdb_movies':
        for record in records:
            title = record.movietitle
            if title is not None:
                movietitle = title.rsplit('(', 1)[0].strip()
            else:
                movietitle = None

            year = record.year

            # Check if the movie has 'France' in moviecountries
            movie_countries = [item.country for item in data['moviecountries'] if item['movieid'] == record.movieid]
            if 'France' in movie_countries:
                data['movies'].append({'movieid': record.movieid, 'movietitle': movietitle, 'year': year})
    elif table_name == 'imdb_actormovies':
        data['actormovies'] = [{'actorid': record.actorid, 'movieid': record.movieid} for record in records]
    elif table_name == 'imdb_directors':
        data['directors'] = [{'directorid': record.directorid, 'directorname': record.directorname} for record in records]
    elif table_name == 'imdb_directormovies':
        data['directormovies'] = [{'directorid': record.directorid, 'movieid': record.movieid} for record in records]
    elif table_name == 'imdb_moviegenres':
        data['moviegenres'] = [{'movieid': record.movieid, 'genre': record.genre} for record in records]
    else:
        print(f'Unknown table {table_name}')
        continue
    print(f'{table_name} processed.')

# Convert actormovies to a dictionary for faster access
actormovies_dict = defaultdict(set)
for am in data['actormovies']:
    actormovies_dict[am['movieid']].add(am['actorid'])

# Process movies
mongodb_movies = []

for movie in data['movies']:
    clear_output(wait=True)
    display(f'Progress: {data["movies"].index(movie) + 1}/{len(data["movies"])}')

    movie_genres = [item['genre'] for item in data['moviegenres'] if item['movieid'] == movie['movieid']]
    movie_directors = [item['directorname'] for item in data['directors'] if item['directorid'] in [dm['directorid'] for dm in data['directormovies'] if dm['movieid'] == movie['movieid']]]
    movie_actors = [item['actorname'] for item in data['actors'] if item['actorid'] in actormovies_dict.get(movie['movieid'], set())]

    most_related_movies = []
    if len(movie_genres) >= 1:
        for other_movie in data['movies']:
            other_movie_genres = [item['genre'] for item in data['moviegenres'] if item['movieid'] == other_movie['movieid']]
            if set(movie_genres) == set(other_movie_genres) and other_movie['movieid'] != movie['movieid']:
                most_related_movies.append({'title': other_movie['movietitle'], 'year': other_movie['year']})
        most_related_movies = sorted(most_related_movies, key=lambda x: x['year'], reverse=True)[:10]

    related_movies = []
    if len(movie_genres) > 1:
        for other_movie in data['movies']:
            other_movie_genres = [item['genre'] for item in data['moviegenres'] if item['movieid'] == other_movie['movieid']]
            if len(set(movie_genres).intersection(set(other_movie_genres))) / len(set(movie_genres)) >= 0.5 and other_movie['movieid'] != movie['movieid'] and other_movie not in most_related_movies:
                related_movies.append({'title': other_movie['movietitle'], 'year': other_movie['year']})

    mongodb_movies.append({
        'title': movie['movietitle'],
        'genres': movie_genres,
        'year': movie['year'],
        'directors': movie_directors,
        'actors': movie_actors,
        'most_related_movies': most_related_movies,
        'related_movies': related_movies
    })
    print(f'title: {movie["movietitle"]}\n genres: {movie_genres}\n year: {movie["year"]}\n directors: {movie_directors}\n actors: {movie_actors}\n most_related_movies: {most_related_movies}\n related_movies: {related_movies}')

Processing imdb_actormovies...
imdb_actormovies processed.
Processing imdb_actors...
imdb_actors processed.
Processing imdb_movies...
imdb_movies processed.
Processing imdb_moviecountries...
imdb_moviecountries processed.
Processing imdb_directors...
imdb_directors processed.
Processing imdb_moviegenres...
imdb_moviegenres processed.
Processing imdb_directormovies...
imdb_directormovies processed.


In [11]:
from pymongo import MongoClient

mongodb_host = 'localhost'
mongodb_port = 27017
mongodb_database = 'si1'
mongodb_collection_name = 'france'

def scifi_from_1994_to_1998(mongodb_collection):
    query = {'year': {'$gte': 1994, '$lte': 1998}, 'genres': 'Sci-Fi'}
    movies = mongodb_collection.find(query)
    for movie in movies:
        print(movie)


def dramas_from_1998_the(mongodb_collection):
    query = {'year': 1998, 'genres': 'Drama', 'movietitle': {'$regex': '^The'}}
    movies = mongodb_collection.find(query)
    for movie in movies:
        print(movie)
  

def faye_dunaway_and_viggo_mortensen(mongodb_collection):
    query = {'actors': {'$all': ['Faye Dunaway', 'Viggo Mortensen']}}
    movies = mongodb_collection.find(query)
    for movie in movies:
        print (movie)

if __name__ == '__main__':
    mongodb_client = MongoClient(mongodb_host, mongodb_port)
    mongodb_db = mongodb_client[mongodb_database]
    mongodb_collection = mongodb_db[mongodb_collection_name]
    
    print('Sci-Fi movies from 1994 to 1998:')
    scifi_from_1994_to_1998(mongodb_collection)
    
    print('\nDramas from 1998 starting with "The":')
    dramas_from_1998_the(mongodb_collection)
    
    print('\nMovies with Faye Dunaway and Viggo Mortensen:')
    faye_dunaway_and_viggo_mortensen(mongodb_collection)
    
    mongodb_client.close()

    

Sci-Fi movies from 1994 to 1998:

Dramas from 1998 starting with "The":

Movies with Faye Dunaway and Viggo Mortensen:


In [4]:
mongodb_db[mongodb_collection_name]

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'si1'), 'france')

In [8]:
query = {'year': {'$gte': 1994, '$lte': 1998}, 'genres': 'Sci-Fi'}
docs  = mongodb_db[mongodb_collection_name].find(query)
for doc in docs:
    print(doc)

In [4]:
from sqlalchemy import create_engine, MetaData, select
from sqlalchemy.orm import sessionmaker
import random
import redis
from sqlalchemy import create_engine, MetaData, select, Table
from sqlalchemy.orm import sessionmaker

postgres_username = 'alumnodb'
postgres_password = 'alumnodb'
postgres_host = 'localhost'
postgres_port = '5432'
postgres_database = 'si1'

def increment_by_email(email):
    key = f"customers:{email}"
    r.hincrby(key, 'visits', 1)

def customer_most_visits():
    keys = r.keys('customers:*')
    max_visits = -1
    max_email = None
    for key in keys:
        visits = int(r.hget(key, 'visits'))
        if visits > max_visits:
            max_visits = visits
            # max_email = key.split(':')[1]
    return max_email

def get_field_by_email(email):
    key = f"customers:{email}"
    name = r.hget(key, 'name')
    phone = r.hget(key, 'phone')
    visits = r.hget(key, 'visits')
    return name, phone, visits


if __name__ == '__main__':
    # Conexión a PostgreSQL
    postgres_uri = f'postgresql://{postgres_username}:{postgres_password}@{postgres_host}:{postgres_port}/{postgres_database}'
    postgres_engine = create_engine(postgres_uri)
    Session = sessionmaker(bind=postgres_engine)
    session = Session()

    # Obtener metadatos de las tablas en PostgreSQL
    metadata = MetaData()
    metadata.reflect(bind=postgres_engine)
    tables_to_process = ['customers']
    
    r = redis.Redis(host='localhost', port=6379, db=0)

    for table_name, table in metadata.tables.items():
        if table_name not in tables_to_process:
            continue
        results = session.query(table).where(table.c.country == 'Spain').all()

    # For each customer, create a hash in Redis
    for result in results:
        key = f"customers:{result.email}"
        name = f"{result.firstname} {result.lastname}"
        phone = result.phone
        visits = random.randint(1, 99)
        r.hset(key, 'name', name)
        r.hset(key, 'phone', phone)
        r.hset(key, 'visits', visits)

In [15]:
def customer_most_visits():
    keys = r.keys('customers:*')
    max_visits = -1
    max_email = None
    for key in keys:
        visits = int(r.hget(key, 'visits'))
        if visits > max_visits:
            max_visits = visits
            max_email = key.decode('utf-8').split(':')[1]
    return max_email

print(customer_most_visits())

liz.taoist@jmail.com


In [18]:
def get_field_by_email(email):
    key = f"customers:{email}"
    name = r.hget(key, 'name').decode('utf-8')
    phone = r.hget(key, 'phone').decode('utf-8')
    visits = int(r.hget(key, 'visits').decode('utf-8'))
    return name, phone, visits

print(get_field_by_email('liz.taoist@jmail.com'))

('liz taoist', '+60 645897838', 99)


In [19]:
def increment_by_email(email):
    key = f"customers:{email}"
    r.hincrby(key, 'visits', 1)
    
increment_by_email('liz.taoist@jmail.com')    
    
print(get_field_by_email('liz.taoist@jmail.com'))

('liz taoist', '+60 645897838', 100)
